In [13]:
# Import Libraries

import pandas as pd
import numpy as np

import sqlite3
from datetime import datetime, timedelta, date
from pmdarima.arima import auto_arima
from fbprophet import Prophet

from config import *

In [1]:
# Setup logging

import logging
logging.basicConfig(filename='crystalball.log', level= logging.ERROR)

In [17]:
# Create DF to Fill Empty Dates - Não Precisa

#def set_hour(date1):
#  day1 = datetime(date1.year, date1.month, date1.day,0,0,0)
#  return day1

#dftime = pd.DataFrame(pd.period_range(start=datetime(2019,11,4,0,0,0), periods=24, freq='7D1H', name = 'Date1'))
#dftime['Date'] = dftime['Date1'].apply(lambda x: set_hour(x))
#dftime = dftime.drop('Date1', 1)
#dftime.set_index('Date', inplace = True)

In [18]:
# Default Data

CPF = '14566178854'


In [2]:
def get_history(cpf):
    try:
        connection = PATH + DATABASE_HISTORY
        sqliteConnection = sqlite3.connect(connection)
        cursor = sqliteConnection.cursor()
        sqlite_select_query = """SELECT * from history_purchase where cpf = """ + cpf
        cursor.execute(sqlite_select_query)
        records = cursor.fetchall()
        records = pd.DataFrame(records)
        records.columns = ['cpf', 'date1', 'prod_id', 'qtde']
        records['date'] =  pd.to_datetime(records['date1'],  format='%Y-%m-%d' )
        records = records.drop('date1', 1)

        cursor.close()
        
        return(records)

    except sqlite3.Error as error:
        logging.error( datetime.now + "Crystalball " + "Failed to read data from sqlite table")

    finally:
        if (sqliteConnection):
            sqliteConnection.close()

In [28]:
##################################################################
# AUTO-ARIMA
##################################################################

def forecast_arima(df):

    # DataFrame for Loop
    dfloop = df.groupby(['prod_id'], as_index=False)['qtde'].count()
    dfloop

    previsoes = pd.DataFrame()
    previsoes['prod_id'] = 1
    previsoes['previsao'] = 2

    #Iteract for product
    for i, r in dfloop.iterrows():
        # Filtrar Item_id em um novo DF
        dffilter = df[df['prod_id'] == r['prod_id']]

        # Se tiver mais que dois registros
        if len(dffilter) > 2:
            # preparar dados para ARIMA
            history = pd.DataFrame()
            history['date'] = pd.to_datetime(dffilter['date'],  format='%Y-%m-%d' )
            history['qtde'] = dffilter['qtde']
            history.set_index('date', inplace = True)

            # AutoArima para ajustas parametros
            stepwise_model = auto_arima(history, start_p=1, start_q=1,max_p=6, max_q=6, m=12, start_P=0, seasonal=True, d=1, D=0, trace=False, error_action='ignore',
                                suppress_warnings=True, stepwise=True)

            #Arima Previsão
            stepwise_model.fit(history)
            forecast = stepwise_model.predict(n_periods=1)

            # Salvar previsão em DF
            print(i+1,'/', len(dfloop) ,r['prod_id'], round(forecast[0],0))
            print()
            df1 = pd.DataFrame({'prod_id': r['prod_id'],  'previsao': [round(forecast[0],0)]})
            previsoes = previsoes.append(df1)

    return previsoes

In [32]:
##################################################################
# Prophet by facebook
##################################################################

def forecast_prophet(df):

    # DataFrame for Loop
    dfloop = df.groupby(['prod_id'], as_index=False)['qtde'].count()
    dfloop

    previsoes = pd.DataFrame()
    previsoes['prod_id'] = 1
    previsoes['previsao'] = 2

    #Iteract for product
    for i, r in dfloop.iterrows():
        # Filtrar Item_id em um novo DF
        dffilter = df[df['prod_id'] == r['prod_id']]

        # Se tiver mais que dois registros
        if len(dffilter) > 2:

            # preparar dados para Prophet
            pdataset = pd.DataFrame()
            pdataset['ds'] = dffilter['date']
            pdataset['y'] = dffilter['qtde']

            #Prophet Previsão e forecast
            prophet = Prophet(growth = 'linear',seasonality_mode='multiplicative' ,yearly_seasonality=False, weekly_seasonality=True, daily_seasonality=False)
            prophet.fit(pdataset)
            #future= prophet.make_future_dataframe(periods=1, freq='w', include_history=False)
            forecast1 = {'ds': ['2020-04-20']}
            future = pd.DataFrame (forecast1)
            forecast=prophet.predict(future)

            # Salvar previsão em DF
            print(i+1,'/', len(dfloop) ,r['prod_id'], round(forecast['yhat'].sum(),0))
            print()
            df1 = pd.DataFrame({'prod_id': r['prod_id'],  'previsao': round(forecast['yhat'],0)})
            previsoes = previsoes.append(df1)

    return previsoes

In [25]:
def forecast(df):
    if FORECAST == 'arima':
        return forecast_arima(df)
    else:
        return forecast_prophet(df)

In [26]:
def cache(df, cpf, date_forecast):
    for i, r in df.iterrows():
        prod_id = r['prod_id']
        previsao = r['previsao']
   
        try:
            connection = PATH + DATABASE_CACHE
            sqliteConnection = sqlite3.connect(connection)
            cursor = sqliteConnection.cursor()
            sql = "INSERT INTO cache ('date', 'cpf', 'prod_id', 'forecast') VALUES ('" + str(date_forecast) + "', "+ str(cpf) + ", " + str(prod_id) + ", " + str(previsao)+ ")"
            cursor.execute(sql)
            sqliteConnection.commit()           

        except sqlite3.Error as error:
            logging.error( datetime.now + "Crystalball " + "Failed to read data from sqlite table")
            return False
        
    sqliteConnection.close()
    return True

    

In [33]:

df = get_history (CPF)
previsoes = forecast(df)
previsoes
retorno = cache(previsoes, CPF, '2020-04-20')
retorno

1 / 11 101 1.0

2 / 11 102 2.0

3 / 11 104 4.0

4 / 11 108 8.0

5 / 11 109 9.0

6 / 11 130 3.0

7 / 11 200 0.0

8 / 11 202 2.0

9 / 11 300 0.0

10 / 11 303 3.0

11 / 11 304 4.0



True